In [1]:
import os
os.chdir('../../scripts/external-data-download/')

In [2]:
OUTPUT_RELATIVE_PATH = "../../data/"
def create_path():
    """
        
    """
    path = ["external-raw-data", "curated/external-data"]
    for target_dir in path:
        if not os.path.exists(OUTPUT_RELATIVE_PATH + target_dir):
            os.makedirs(OUTPUT_RELATIVE_PATH + target_dir)
    print('Already Create Paths')

create_path()

Already Create Paths


In [3]:
from download import download

download.download_all()

/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:99: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hospital_df['geometry'] = hospital_df['geometry'].centroid
/Users/quzihanwu/Desktop/generic-real-estate-consulting-project-group-7/scripts/external-data-download/download.py:124: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  emergency_service_df['geometry'] = emergency_service_df['geometry'].centroid
